## LLAMA 70B

1. Choose the following kernel: PyTorch-2.0.1
2. Install accelerate and trnasformers by running the below cell
3. You can find more information about LLAMA 70B: https://huggingface.co/upstage/SOLAR-0-70b-16bit

**Please do not edit any code**

In [1]:
!pip install accelerate sentencepiece jsonformer 
!pip3 install git+https://github.com/huggingface/transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /scratch/local/13736565/pip-req-build-5bgrgkde
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /scratch/local/13736565/pip-req-build-5bgrgkde
  Resolved https://github.com/huggingface/transformers to commit 6f316016877197014193b9463b2fd39fa8f0c8e4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from huggingface_hub import notebook_login
notebook_login()


In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import transformers
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoConfig,AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer
import json
from jsonformer import Jsonformer

In [4]:
path_to_model="/blue/cap4773/sosa.s/Llama-2-7b-chat-hf"
device = "cuda"
tokenizer = LlamaTokenizer.from_pretrained(path_to_model)
model = LlamaForCausalLM.from_pretrained(path_to_model, torch_dtype=torch.float16, device_map=device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
current_functions = [
    {
        "name": "row_orchestration",
        "description": "Determine the type of row UI and product to show",
        "parameters": {
            "type": "object",
            "properties": {
                "prod_type":
                {
                    "type": "string",
                    "enum": ["plans", "devices", "information", "help"],
                    "description": "The service, product, or assistance inside the container the client needs",
                },
                "row_type": {
                    "type": "string",
                    "enum": ["table", "cards"],
                    "description": "Choose table if user wants to compare prod_type and cards for exploration prod_type",
                },

            },
            "required": ["row_type", "prod_type"],
        },
    }
]
prompt = """<s>[INST] <<SYS>>
When returning in function format you must begin with <FUNCTION_CALL> with no context
<</SYS>>

<FUNCTIONS>[
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
What is the weather like in Boston? [/INST]
<FUNCTION_CALL>{"name": "get_current_weather", "arguments": "{ \\"location\\": \\"Boston, MA\\"}"}</FUNCTION_CALL></s>
<s>[INST] <FUNCTIONS>[
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
Hi how are you? [/INST]
I am doing well, thank you for asking.</s>
<s>[INST] <FUNCTIONS>""" + json.dumps(current_functions) + """
Show me devices to compare [/INST]"""

In [19]:
# jsonformer
current_prompt = prompt.replace("[SYSTEM_MESSAGE]", "When returning in function format you must respond like so: <FUNCTION_CALL>[NAME_OF_FUNCTION]<FUNCTION_CALL>")
inputs = tokenizer.encode(current_prompt, return_tensors="pt").to(device)

outputs = model.generate(inputs, max_new_tokens=500)
if int(outputs[0][0]) == 1 and int(outputs[0][1]) == 1:
     outputs = outputs[:, 1:]

generated_string = tokenizer.decode(outputs[0])[len(current_prompt):].strip()
print("Generated: ", generated_string)

Generated:  <FUNCTION_CALL>{"name": "row_orchestration", "arguments": {"type": {"prod_type": "devices", "row_type": "cards"}}}</FUNCTION_CALL></s>


In [20]:
def extract_function_name(generated_string):
    """
    Extracts the function name from the generated string.

    :param generated_string: The generated string containing the function call
    :return: The function name as a string, or None if not found
    """
    # Strip <FUNCTION_CALL> tags and any surrounding whitespace
    if ("<FUNCTION_CALL>" not in generated_string or "</FUNCTION_CALL>" not in generated_string):
        print("Error: Not a function call")
        return None
        
    stripped_string = generated_string.replace("<FUNCTION_CALL>", "").replace("</FUNCTION_CALL>", "").strip()

    # Find the start and end of the "name" value
    name_key = '"name":'
    start_idx = stripped_string.find(name_key)
    if start_idx == -1:
        print("Error: 'name' key not found.")
        return None
    
    # Adjust start index to the beginning of the value
    start_idx += len(name_key)
    
    # Find the start and end quotes of the value
    start_quote = stripped_string.find('"', start_idx)
    end_quote = stripped_string.find('"', start_quote + 1)
    
    if start_quote == -1 or end_quote == -1:
        print("Error: Unable to find the value for 'name'.")
        return None
    
    # Extract the function name
    function_name = stripped_string[start_quote + 1:end_quote]

    return function_name

# Example usage
_generated_string = '   <FUNCTION_CALL>{"name": "row_orchestration", "arguments": {"row_type": "cards", "prod_type": "devices"}}</FUNCTION_CALL></s>'
_function_name = extract_function_name(_generated_string)
if _function_name:
    print("Function Name:", _function_name)


Function Name: row_orchestration


In [21]:
function_name = extract_function_name(generated_string)

if (function_name):
    function_to_use = next((func for func in current_functions if func["name"] == function_name), None)
    jsonformer = Jsonformer(model, tokenizer, function_to_use["parameters"], prompt)
    generated_string = "<FUNCTION_CALL>" + json.dumps(jsonformer())+ "</FUNCTION_CALL></s>"

generated_string

'<FUNCTION_CALL>{"prod_type": "devices", "row_type": "table"}</FUNCTION_CALL></s>'